In [4]:
!pip install imutils 

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25859 sha256=38aa9b448aa506cc49435933b0c472fd9cbcfb637bb80995b354986e8e9bec95
  Stored in directory: /root/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imutils import paths
import os
list_train = list(paths.list_images('/kaggle/input/butterfly-image-classification/train'))
list_test = list(paths.list_images('/kaggle/input/butterfly-image-classification/test'))
train_set = pd.read_csv('/kaggle/input/butterfly-image-classification/Training_set.csv')
test_set = pd.read_csv('/kaggle/input/butterfly-image-classification/Testing_set.csv')

In [4]:
train_set.nunique()

filename    6499
label         75
dtype: int64

In [5]:
print(len(list_train))
print(train_set.shape)
print(len(list_test))
print(test_set.shape)

6499
(6499, 2)
2786
(2786, 1)


In [6]:
print(train_set.head())
print(test_set.head())

      filename                     label
0  Image_1.jpg          SOUTHERN DOGFACE
1  Image_2.jpg                    ADONIS
2  Image_3.jpg            BROWN SIPROETA
3  Image_4.jpg                   MONARCH
4  Image_5.jpg  GREEN CELLED CATTLEHEART
      filename
0  Image_1.jpg
1  Image_2.jpg
2  Image_3.jpg
3  Image_4.jpg
4  Image_5.jpg


# Train models

## Model1 SVM

In [7]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelEncoder

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [8]:
train1 = []
labels1 = []
for i in list_train:
    label = train_set[train_set['filename'] == i.split(os.path.sep)[-1]]['label'].values[0]
    img = load_img(i,target_size = (32,32,3))
    img = img_to_array(img)
    img = img.flatten()
    train1.append(img)
    labels1.append(label)
train1 = np.array(train1,dtype = 'float32')  
labels1 = np.array(labels1)
enc = LabelEncoder().fit(labels1)
labels1 = enc.transform(labels1)

In [9]:
test_data1 = []
test_label = []
for i in list_test:
    label = i.split(os.path.sep)[-1]
    img = load_img(i,target_size = (32,32,3))
    img = img_to_array(img)
    img = img.flatten()
    test_data1.append(img)
    test_label.append(label)
test_data1 = np.array(test_data1,dtype='float32')/255
test_label = np.array(test_label)

In [10]:
test_label = pd.DataFrame({'filename':test_label})

In [11]:
print(train1[:5])
print(labels1[:5])

[[ 30.  42.  18. ...  24.  31.  15.]
 [118. 114.  69. ... 187. 212. 110.]
 [157. 161.  38. ... 231. 220. 200.]
 [174. 149. 118. ...  85.  81.  80.]
 [119. 155.  68. ... 166. 205. 124.]]
[20 21 11  8 33]


In [12]:
df1 = pd.DataFrame(train1)
df1['target'] = labels1
x = df1.iloc[:,:-1]/255
y = df1.iloc[:,-1]

In [13]:
x.head()

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,0.117647,0.164706,0.070588,0.113725,0.160784,0.066667,0.109804,0.156863,0.070588,0.101961,...,0.050980,0.086275,0.113725,0.050980,0.090196,0.117647,0.054902,0.094118,0.121569,0.058824
1,0.462745,0.447059,0.270588,0.466667,0.435294,0.290196,0.458824,0.419608,0.282353,0.454902,...,0.482353,0.827451,0.909804,0.552941,0.745098,0.831373,0.439216,0.733333,0.831373,0.431373
2,0.615686,0.631373,0.149020,0.619608,0.639216,0.145098,0.643137,0.654902,0.149020,0.654902,...,0.741176,0.937255,0.858824,0.760784,0.862745,0.784314,0.686275,0.905882,0.862745,0.784314
3,0.682353,0.584314,0.462745,0.854902,0.784314,0.776471,0.784314,0.713726,0.698039,0.580392,...,0.062745,0.705882,0.690196,0.694118,0.572549,0.545098,0.572549,0.333333,0.317647,0.313726
4,0.466667,0.607843,0.266667,0.478431,0.615686,0.286275,0.482353,0.623529,0.282353,0.470588,...,0.121569,0.435294,0.545098,0.254902,0.607843,0.745098,0.415686,0.650980,0.803922,0.486275


In [14]:
from sklearn.model_selection import train_test_split
X_train1,X_valid1,y_train1,y_valid1 = train_test_split(x, y, test_size = 0.2, random_state = 27,stratify = y) 

In [15]:
pars = {'C' : [100],
       'gamma' : [0.0001],
       'kernel' : ['rbf']}
svc = svm.SVC(probability = True)
model1 = GridSearchCV(svc,pars,verbose=10)

In [16]:
model1.fit(X_train1,y_train1)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START C=100, gamma=0.0001, kernel=rbf.............................
[CV 1/5; 1/1] END C=100, gamma=0.0001, kernel=rbf;, score=0.338 total time= 3.0min
[CV 2/5; 1/1] START C=100, gamma=0.0001, kernel=rbf.............................
[CV 2/5; 1/1] END C=100, gamma=0.0001, kernel=rbf;, score=0.362 total time= 3.0min
[CV 3/5; 1/1] START C=100, gamma=0.0001, kernel=rbf.............................
[CV 3/5; 1/1] END C=100, gamma=0.0001, kernel=rbf;, score=0.346 total time= 3.0min
[CV 4/5; 1/1] START C=100, gamma=0.0001, kernel=rbf.............................
[CV 4/5; 1/1] END C=100, gamma=0.0001, kernel=rbf;, score=0.330 total time= 3.0min
[CV 5/5; 1/1] START C=100, gamma=0.0001, kernel=rbf.............................
[CV 5/5; 1/1] END C=100, gamma=0.0001, kernel=rbf;, score=0.341 total time= 3.0min


GridSearchCV(estimator=SVC(probability=True),
             param_grid={'C': [100], 'gamma': [0.0001], 'kernel': ['rbf']},
             verbose=10)

In [17]:
pred1 = model1.predict(X_valid1)

In [18]:
print(model1.best_params_)

{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}


In [20]:
print(pred1[:5])

[11 17 41 10 54]


In [21]:
print(accuracy_score(y_valid1,pred1))
print(classification_report(y_valid1,pred1))

0.3446153846153846
              precision    recall  f1-score   support

           0       0.71      0.56      0.63        18
           1       0.82      0.60      0.69        15
           2       0.10      0.13      0.11        15
           3       0.62      0.29      0.40        17
           4       0.48      0.67      0.56        18
           5       0.23      0.45      0.31        20
           6       0.32      0.50      0.39        20
           7       0.28      0.29      0.29        17
           8       0.19      0.25      0.22        16
           9       0.14      0.24      0.18        17
          10       0.57      0.53      0.55        15
          11       0.14      0.18      0.16        17
          12       0.28      0.25      0.26        20
          13       0.32      0.44      0.37        18
          14       0.53      0.59      0.56        17
          15       0.27      0.37      0.31        19
          16       0.36      0.24      0.29        17
        

In [22]:
pred_test1 = model1.predict_proba(test_data1)

## Model KNN

In [23]:
from sklearn.neighbors import KNeighborsClassifier
par2 = {'n_neighbors': [1,2]}
kneig = KNeighborsClassifier()
model2 = GridSearchCV(kneig,par2,verbose=10)
model2.fit(X_train1,y_train1)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5; 1/2] START n_neighbors=1...............................................
[CV 1/5; 1/2] END ................n_neighbors=1;, score=0.167 total time=   0.6s
[CV 2/5; 1/2] START n_neighbors=1...............................................
[CV 2/5; 1/2] END ................n_neighbors=1;, score=0.172 total time=   0.4s
[CV 3/5; 1/2] START n_neighbors=1...............................................
[CV 3/5; 1/2] END ................n_neighbors=1;, score=0.155 total time=   0.4s
[CV 4/5; 1/2] START n_neighbors=1...............................................
[CV 4/5; 1/2] END ................n_neighbors=1;, score=0.161 total time=   0.4s
[CV 5/5; 1/2] START n_neighbors=1...............................................
[CV 5/5; 1/2] END ................n_neighbors=1;, score=0.167 total time=   0.4s
[CV 1/5; 2/2] START n_neighbors=2...............................................
[CV 1/5; 2/2] END ................n_neighbors=2;,

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2]}, verbose=10)

In [24]:
pred2 = model2.predict(X_valid1)

In [25]:
print(model2.best_params_)

{'n_neighbors': 1}


In [26]:
print(pred2[:5])

[12 41 64 10 54]


In [27]:
print(accuracy_score(y_valid1,pred2))
print(classification_report(y_valid1,pred2))

0.16307692307692306
              precision    recall  f1-score   support

           0       0.86      0.33      0.48        18
           1       0.67      0.40      0.50        15
           2       0.00      0.00      0.00        15
           3       1.00      0.12      0.21        17
           4       0.00      0.00      0.00        18
           5       0.28      0.25      0.26        20
           6       0.62      0.25      0.36        20
           7       0.50      0.18      0.26        17
           8       0.07      0.12      0.09        16
           9       0.03      0.24      0.06        17
          10       0.50      0.40      0.44        15
          11       0.07      0.18      0.10        17
          12       0.23      0.15      0.18        20
          13       0.22      0.22      0.22        18
          14       0.50      0.18      0.26        17
          15       0.06      0.16      0.09        19
          16       0.67      0.12      0.20        17
       

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
pred_test2 = model2.predict_proba(test_data1)

## Model Random Forest

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
par3 = {'n_estimators':[500],
       'max_features':['sqrt'],
       'max_depth':[19],
       'criterion':['gini']}

In [31]:
rfc = RandomForestClassifier(random_state=27)
model3 = GridSearchCV(rfc,par3,verbose=10)
model3.fit(X_train1,y_train1)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500
[CV 1/5; 1/1] END criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500;, score=0.364 total time= 2.6min
[CV 2/5; 1/1] START criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500
[CV 2/5; 1/1] END criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500;, score=0.365 total time= 2.6min
[CV 3/5; 1/1] START criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500
[CV 3/5; 1/1] END criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500;, score=0.348 total time= 2.6min
[CV 4/5; 1/1] START criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500
[CV 4/5; 1/1] END criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500;, score=0.358 total time= 2.6min
[CV 5/5; 1/1] START criterion=gini, max_depth=19, max_features=sqrt, n_estimators=500
[CV 5/5; 1/1] END criterion=gini, max_depth

GridSearchCV(estimator=RandomForestClassifier(random_state=27),
             param_grid={'criterion': ['gini'], 'max_depth': [19],
                         'max_features': ['sqrt'], 'n_estimators': [500]},
             verbose=10)

In [32]:
pred3 = model3.predict(X_valid1)

In [33]:
print(pred3[:5])

[12 17 41 70 54]


In [34]:
print(model3.best_params_)

{'criterion': 'gini', 'max_depth': 19, 'max_features': 'sqrt', 'n_estimators': 500}


In [35]:
print(accuracy_score(y_valid1,pred3))
print(classification_report(y_valid1,pred3))

0.3384615384615385
              precision    recall  f1-score   support

           0       0.45      0.56      0.50        18
           1       0.50      0.60      0.55        15
           2       0.25      0.07      0.11        15
           3       0.75      0.35      0.48        17
           4       0.59      0.56      0.57        18
           5       0.22      0.60      0.32        20
           6       0.36      0.60      0.45        20
           7       0.62      0.29      0.40        17
           8       0.11      0.12      0.11        16
           9       0.15      0.47      0.23        17
          10       0.80      0.27      0.40        15
          11       0.00      0.00      0.00        17
          12       0.28      0.25      0.26        20
          13       0.38      0.56      0.45        18
          14       0.50      0.41      0.45        17
          15       0.10      0.11      0.10        19
          16       0.20      0.06      0.09        17
        

In [36]:
pred_test3 = model3.predict_proba(test_data1)

## Model Ensembled

In [ ]:
#pred123 = (pred_test1 + pred_test2 + pred_test3)/3

In [ ]:
#pred123 = np.argmax(pred123,axis = 1)

In [ ]:
#test_label['label'] = enc.inverse_transform(pred123)

In [ ]:
#submission = pd.merge(test_set,test_label,how = 'left',on = 'filename')

In [ ]:
#submission.to_csv('Testing_set.csv',index = False)

In [ ]:
#submission.head()

## Model CNN DenseNet121

In [6]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
train_data = []
train_label = []
for i in list_train:
    label = train_set[train_set['filename']==i.split(os.path.sep)[-1]]['label'].values[0]
    image = load_img(i,target_size = (224,224,3))
    image = img_to_array(image)
    train_data.append(image)
    train_label.append(label)
train_data = np.array(train_data,dtype='float32')
train_label = np.array(train_label)  

In [7]:
test_data = []
test_label = []
for i in list_test:
    label = i.split(os.path.sep)[-1]
    image = load_img(i,target_size = (224,224,3))
    image = img_to_array(image)
    test_data.append(image)
    test_label.append(label)
test_data = np.array(test_data,dtype='float32')
test_label = np.array(test_label)

In [8]:
test_label = pd.DataFrame({'filename':test_label})

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import AveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import math

In [10]:
enc = LabelEncoder()
enc.fit(train_label)
train_label = enc.transform(train_label)
print(train_label[:2])

[20 21]


In [11]:
train_label = tf.keras.utils.to_categorical(train_label,num_classes = 75)
print(train_label[:2])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]]


In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_label, test_size = 0.2, random_state = 27, stratify = train_label)

In [13]:
print(len(X_train))
print(len(X_valid))
print(len(y_train))
print(len(y_valid))

5199
1300
5199
1300


In [14]:
gen = ImageDataGenerator(zoom_range = 0.2,
                        height_shift_range = 0.2,
                        width_shift_range = 0.2,
                        rotation_range = 20,
                        shear_range = 0.2,
                        horizontal_flip = True,
                        vertical_flip = True,
                        fill_mode = 'nearest'
                        )

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
ckp = ModelCheckpoint("model.keras", save_best_only = True)
est = EarlyStopping(monitor = "val_loss", patience = 13)
red = ReduceLROnPlateau(monitor = 'val_loss', min_lr = 0.0000001, factor = 0.1, patience = 3)

In [16]:
class GeM(tf.keras.layers.Layer):
    def __init__(self, pool_size, init_norm=3.0, normalize=False, **kwargs):
        self.pool_size = pool_size
        self.init_norm = init_norm
        self.normalize = normalize

        super(GeM, self).__init__(**kwargs)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'pool_size': self.pool_size,
            'init_norm': self.init_norm,
            'normalize': self.normalize,
        })
        return config

    def build(self, input_shape):
        feature_size = input_shape[-1]
        self.p = self.add_weight(name='norms', shape=(feature_size,),
                                 initializer=tf.keras.initializers.constant(self.init_norm),
                                 trainable=True)
        super(GeM, self).build(input_shape)

    def call(self, inputs):
        x = inputs
        x = tf.math.maximum(x, 1e-6)
        x = tf.pow(x, self.p)

        x = tf.nn.avg_pool(x, self.pool_size, self.pool_size, 'VALID')
        x = tf.pow(x, 1.0 / self.p)

        if self.normalize:
            x = tf.nn.l2_normalize(x, 1)
        return x

    def compute_output_shape(self, input_shape):
        return tuple([None, input_shape[-1]])

In [17]:
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )#(1,512)*(512,81313)
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)#(1,81313)*(1,81313) = (1,81313)
        output *= self.s
        return output

In [18]:
from tensorflow.keras.utils import Sequence
class CustomDataGenerator(Sequence):
    def __init__(self, features, labels, batch_size):
        self.features = features
        self.labels = labels
        self.batch_size = batch_size
    def __len__(self):
        return len(self.features)//self.batch_size
    def __getitem__(self,index):
        features_batch = self.features[self.batch_size*index:self.batch_size*(index+1)]
        labels_batch = self.labels[self.batch_size*index:self.batch_size*(index+1)]
        input2 = np.argmax(labels_batch,axis = 1)
        return (features_batch, input2), labels_batch

In [19]:
train = CustomDataGenerator(X_train,y_train,64)
valid = CustomDataGenerator(X_valid,y_valid,64)

In [20]:
label = Input(shape = ())
basemodel1 = DenseNet121(weights = 'imagenet',include_top = False, input_tensor = Input(shape = (224,224,3)))
headmodel1 = basemodel1.output
headmodel1 = GeM(7)(headmodel1)
headmodel1 = Flatten()(headmodel1)
headmodel1 = Dense(units=512,activation = 'relu')(headmodel1)
headmodel1 = BatchNormalization(name="dense_before_arcface")(headmodel1)
headmodel1 = ArcMarginProduct(n_classes = 75)([headmodel1,label])
headmodel1 = Dense(units = 75,activation = 'softmax')(headmodel1)
model_densenet = Model(inputs = [basemodel1.input,label], outputs = headmodel1)

29084464/29084464 [==============================] - 2s 0us/step


In [21]:
for i in basemodel1.layers:
    i.trainable = False

In [23]:
model_densenet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                           

In [23]:
lr = 0.0001
Epochs = 99
batch_size = 64
import tensorflow as tf
import keras
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate = lr,
                decay_steps = 10000,
                decay_rate = 0.9)
opt = Adam(learning_rate = lr_schedule)
model_densenet.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
#keras.metrics.AUC(name = 'auc')
model_densenet.fit(train,
          validation_data = valid,
         epochs = Epochs)

Epoch 1/99
81/81 [==============================] - 21s 155ms/step - loss: 5.7229 - accuracy: 0.0424 - val_loss: 5.3387 - val_accuracy: 0.0562
Epoch 2/99
81/81 [==============================] - 11s 133ms/step - loss: 4.6314 - accuracy: 0.1154 - val_loss: 4.5619 - val_accuracy: 0.1281
Epoch 3/99
81/81 [==============================] - 11s 133ms/step - loss: 3.9668 - accuracy: 0.1892 - val_loss: 4.0565 - val_accuracy: 0.1914
Epoch 4/99
81/81 [==============================] - 11s 134ms/step - loss: 3.4605 - accuracy: 0.2483 - val_loss: 3.6827 - val_accuracy: 0.2273
Epoch 5/99
81/81 [==============================] - 11s 133ms/step - loss: 3.0337 - accuracy: 0.3229 - val_loss: 3.3594 - val_accuracy: 0.2672
Epoch 6/99
81/81 [==============================] - 11s 133ms/step - loss: 2.6843 - accuracy: 0.3804 - val_loss: 3.1044 - val_accuracy: 0.3117
Epoch 7/99
81/81 [==============================] - 11s 132ms/step - loss: 2.3712 - accuracy: 0.4361 - val_loss: 2.8955 - val_accuracy: 0.3508

In [24]:
preds = np.argmax(model_densenet.predict(valid),axis = 1)
print(preds[:10])
print(np.argmax(y_valid[:10],axis = 1))

20/20 [==============================] - 4s 98ms/step
[34 65 17  4  4 30 10 40 27 54]
[47 65 17 43  4 30 10 40 27 54]


In [27]:
modelbase_inference = Model(inputs=basemodel1.input,
                outputs = model_densenet.get_layer("dense_before_arcface").output)
out = modelbase_inference.output
out = Dense(units=512,activation = 'relu')(out)
out = Dropout(rate = 0.5)(out)
out = Dense(units=128,activation = 'relu')(out)
out = Dense(units = 75,activation = 'softmax')(out)
model_inference = Model(inputs = basemodel1.input, outputs = out)
for i in modelbase_inference.layers:
    i.trainable = False


In [28]:
lr = 0.0001
Epochs = 40
batch_size = 64
import tensorflow as tf
import keras
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate = lr,
                decay_steps = 10000,
                decay_rate = 0.9)
opt = Adam(learning_rate = lr_schedule)
model_inference.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
model_inference.fit(gen.flow(X_train,y_train,batch_size = batch_size),
          steps_per_epoch = len(X_train)//batch_size,
          validation_data = (X_valid,y_valid),
          validation_steps = len(X_valid)/batch_size,
         epochs = Epochs)

Epoch 1/40
81/81 [==============================] - 65s 723ms/step - loss: 4.5817 - accuracy: 0.0175 - val_loss: 4.2129 - val_accuracy: 0.0385
Epoch 2/40
81/81 [==============================] - 55s 677ms/step - loss: 4.2532 - accuracy: 0.0425 - val_loss: 4.0193 - val_accuracy: 0.0985
Epoch 3/40
81/81 [==============================] - 55s 674ms/step - loss: 4.0397 - accuracy: 0.0804 - val_loss: 3.8300 - val_accuracy: 0.1631
Epoch 4/40
81/81 [==============================] - 54s 669ms/step - loss: 3.8542 - accuracy: 0.1264 - val_loss: 3.6168 - val_accuracy: 0.2262
Epoch 5/40
81/81 [==============================] - 55s 673ms/step - loss: 3.6622 - accuracy: 0.1675 - val_loss: 3.3858 - val_accuracy: 0.2823
Epoch 6/40
81/81 [==============================] - 55s 675ms/step - loss: 3.4083 - accuracy: 0.2105 - val_loss: 3.1459 - val_accuracy: 0.3300
Epoch 7/40
81/81 [==============================] - 55s 677ms/step - loss: 3.2624 - accuracy: 0.2325 - val_loss: 2.9247 - val_accuracy: 0.3692

In [30]:
preds = np.argmax(model_inference.predict(X_valid[:10]),axis = 1)
print(preds[:10])
print(np.argmax(y_valid[:10],axis = 1))

1/1 [==============================] - 3s 3s/step
[47 11 17  4  4 30 10 62 27 54]
[47 65 17 43  4 30 10 40 27 54]


## Model CNN EfficientNetB0

In [ ]:
basemodel = EfficientNetB0(weights = 'imagenet',include_top = False, input_tensor = Input(shape = (224,224,3)))
headmodel = basemodel.output
headmodel = AveragePooling2D()(headmodel)
headmodel = Flatten()(headmodel)
headmodel = Dense(units=512,activation = 'relu')(headmodel)
headmodel = Dropout(rate = 0.5)(headmodel)
headmodel = Dense(units=128,activation = 'relu')(headmodel)
headmodel = Dense(units = 75,activation = 'softmax')(headmodel)
model = Model(inputs = basemodel.input, outputs = headmodel)

In [ ]:
for i in basemodel.layers:
    i.trainable = False

In [ ]:
lr = 0.0001
Epochs = 40
batch_size = 64
import tensorflow as tf
import keras
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate = lr,
                decay_steps = 10000,
                decay_rate = 0.9)
opt = Adam(learning_rate = lr_schedule)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
model.fit(gen.flow(X_train,y_train,batch_size = batch_size),
          steps_per_epoch = len(X_train)//batch_size,
          validation_data = (X_valid,y_valid),
          validation_steps = len(X_valid)/batch_size,
         epochs = Epochs)

## Model CNN EfficientNetV2**

In [ ]:
basemodel2 = EfficientNetV2L(weights = 'imagenet',include_top = False, input_tensor = Input(shape = (224,224,3)))
headmodel2 = basemodel2.output
headmodel2 = AveragePooling2D()(headmodel2)
headmodel2 = Flatten()(headmodel2)
headmodel2 = Dense(units=128,activation = 'relu')(headmodel2)
headmodel2 = Dropout(rate = 0.5)(headmodel2)
headmodel2 = Dense(units=128,activation = 'relu')(headmodel2)
headmodel2 = Dense(units = 75,activation = 'softmax')(headmodel2)
model_v2 = Model(inputs = basemodel2.input, outputs = headmodel2)

In [ ]:
for i in basemodel2.layers:
    i.trainable = False

In [ ]:
lr = 0.0001
Epochs = 8
batch_size = 64
import tensorflow as tf
import keras
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate = lr,
                decay_steps = 10000,
                decay_rate = 0.9)
opt = Adam(learning_rate = lr_schedule)
model_v2.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = [keras.metrics.AUC(name = 'auc')])
model_v2.fit(gen.flow(X_train,y_train,batch_size = batch_size),
          steps_per_epoch = len(X_train)//batch_size,
          validation_data = (X_valid,y_valid),
          validation_steps = len(X_valid)/batch_size,
         epochs = Epochs)

# Test data

In [ ]:
# = model.predict(test_data)

In [ ]:
#print(predict[:10])

In [ ]:
#predict = np.argmax(predict,axis = 1)

In [ ]:
#predict[:5]

In [ ]:
#test_label['label'] = enc.inverse_transform(predict)

In [ ]:
#test_label

In [ ]:
#test_label[test_label['filename']=='Image_1.jpg']

In [ ]:
#submission = pd.merge(test_set,test_label,how = 'left',on = 'filename')

In [ ]:
#print(test_set.shape)
#print(submission.shape)

In [ ]:
#submission.head()

In [ ]:
#submission.to_csv('Testing_set.csv',index = False)

In [ ]:
predict1 = 0.2 * model_v2.predict(test_data) + 0.6 * model.predict(test_data) \
+ 0.05 * model_densenet.predict(test_data) + 0.05 * model1.predict_proba(test_data1) \
+ 0.05 * model2.predict_proba(test_data1) + 0.05 * model3.predict_proba(test_data1)
predict1 = np.argmax(predict1,axis = 1)
test_label['label'] = enc.inverse_transform(predict1)
submission1 = pd.merge(test_set,test_label,how = 'left',on = 'filename')
submission1.to_csv('Testing_set1.csv',index = False)